# DATA CLEANING 

# fns to import statements

In [60]:
# imports 
import pandas as pd
import numpy as np 
import yfinance as yf 
from yahoofinancials import YahooFinancials

def getIncomeStatement(ticker, yahoo_financials):
    """
        Function params: string ticker (the ticker of the company of choice)
        Returns: A data frame of the quartler income statement for the company
    """
    # import data
    # yahoo_financials = YahooFinancials('AAPL')
    data = yahoo_financials.get_financial_stmts('quarterly', 'income')
    # Drill down in dictionary to get the data we want 
    dict_list = data['incomeStatementHistoryQuarterly'][ticker]
    # Create the dataframe
    df = pd.concat([pd.DataFrame(i) for i in dict_list], axis=1)
    # Sort Columns 
    df = df.reindex(sorted(df.columns), axis=1)
    # rename the index
    df = df.rename_axis('date').reset_index()
    df = df.transpose()
    return df

def getBalanceSheet(ticker, yahoo_financials):
    """
        Function params: string ticker (the ticker of the company of choice)
        Returns: A data frame of the quartler balance for the company 
    """
    # import data
    # yahoo_financials = YahooFinancials(ticker)
    data = yahoo_financials.get_financial_stmts('quarterly', 'balance')
    # print(data)
    # Drill down in dictionary to get the data we want 
    dict_list = data['balanceSheetHistoryQuarterly'][ticker]
    # Create the dataframe
    df = pd.concat([pd.DataFrame(i) for i in dict_list], axis=1)
    # Sort Columns 
    df = df.reindex(sorted(df.columns), axis=1)
    # rename the index
    df = df.rename_axis('date').reset_index()
    df = df.transpose()
    return df

def getCashSheet(ticker, yahoo_financials):
    """
        Function params: string ticker (the ticker of the company of choice)
        Returns: A data frame of the quartler balance for the company 
    """
    # import data
    # yahoo_financials = YahooFinancials(ticker)
    data = yahoo_financials.get_financial_stmts('quarterly', 'cash')
    # Drill down in dictionary to get the data we want 
    dict_list = data['cashflowStatementHistoryQuarterly'][ticker]
    # Create the dataframe
    df = pd.concat([pd.DataFrame(i) for i in dict_list], axis=1)
    # Sort Columns 
    df = df.reindex(sorted(df.columns), axis=1)
    # rename the index
    df = df.rename_axis('date').reset_index()
    df = df.transpose()
    return df

def getPrices(ticker, yahoo_financials):
    """
    Inputs: ticker of a company 
    Outputs: the prices of the 4 quarterly statements for 2022 
    NOTE: price is technically for 12/30 bc there was no price available for 12/31
    """
    # yahoo_financials = YahooFinancials(ticker)
    lst_dates_interst = ['2022-03-31', '2022-06-30','2022-09-30', '2022-12-30']
    lst_end_dates = ['2022-4-1', '2022-7-1', '2022-10-1', '2023-1-2']

    # get first data so that it can be concated later on 
    data = yahoo_financials.get_historical_price_data(start_date=lst_dates_interst[0], end_date=lst_end_dates[0], time_interval='daily')
    df = pd.DataFrame(data[ticker]['prices'])

    # go through and get the prices for each data and add it to the df
    for i in range(len(lst_dates_interst)-1):
        data = yahoo_financials.get_historical_price_data(start_date=lst_dates_interst[i+1], end_date=lst_end_dates[i+1], time_interval='daily')
        temp = pd.DataFrame(data[ticker]['prices'])
        df = pd.concat([df, temp], axis=0)
    # Set dates to be axis. Note price is technically for 12/30 bc there was no price available for 12/31
    df = df.set_axis(['2022-03-31', '2022-06-30', '2022-09-30', '2022-12-31'], axis='index')

    # get rid of date columns because they're now the index
    df = df.drop(columns=['formatted_date', 'date'], axis=0)
    return df

# fns to extract metrics of interest

In [61]:
# create functions that extract metrics 

def clean_df(df):
    """
    Inputs: a 'raw' dataframe 
    Outputs: a dataframe that has actually column names and gets rid of unneccsary columns
    NOTE: this function is called by each of the three function below before extracting metrics.  
    """
    cols = df.iloc[0]
    df = df.rename(columns=cols)
    df = df.drop(['date'])
    return df

def getIncomeMetrics (df):
    """
    Inputs: a 'raw' dataframe 
    Outputs: a dataframe that contains only the metrics of interest
    """
    df = clean_df(df)
    income_metrics = [
        'dilutedNIAvailtoComStockholders',
        'grossProfit',
        'netIncome',
        'totalExpenses',
        'totalRevenue',
        'basicEPS', 
    ]
    df2 = pd.DataFrame([])
    for metric in income_metrics:
        if metric in df.columns:
            # get the data from df and put it in the correct column in df2
            df2[metric] = df[metric]
        else:
            # fill the data with NA
            df2[metric] = pd.NA
    return df2

def getBalanceSheetMetrics (df):
    """
    Inputs: a 'raw' dataframe 
    Outputs: a dataframe that contains only the metrics of interest
    """
    df = clean_df(df)
    balance_metrics = [
    'cashEquivalents',
    'currentAssets',
    'currentDebt',
    'netDebt',
    'totalCapitalization',
    'totalDebt',
    'totalAssets',
    'tangibleBookValue',
    'shareIssued'
    ]
    df2 = pd.DataFrame([])
    for metric in balance_metrics:
        if metric in df.columns:
            # get the data from df and put it in the correct column in df2
            df2[metric] = df[metric]
        else:
            # fill the data with NA
            df2[metric] = pd.NA
    return df2

def getCashMetrics (df): 
    """
    Inputs: a 'raw' dataframe 
    Outputs: a dataframe that contains only the metrics of interest
    """
    df = clean_df(df)
    cash_metrics = [
    'freeCashFlow',
    ]
    df2 = pd.DataFrame([])
    for metric in cash_metrics:
        if metric in df.columns:
            # get the data from df and put it in the correct column in df2
            df2[metric] = df[metric]
        else:
            # fill the data with NA
            df2[metric] = pd.NA
    return df2

# fns to Calculate Metrics 

In [62]:
def marketCap (price, sharesIssued):
    return price * sharesIssued

def pricePerEarningsPerShare (price, eps):
    return price/eps

def priceToBook (price, netAssets):
    return price / netAssets

In [63]:
def getCalculatedMetrics(df):
    """
    Inputs: Takes a df 
    Outputs: The same dataframe with columns added that contian the metrics. 
    """
    # empty lists to store the metrics 
    temp_market, temp_price_per_eps, temp_price_to_book =  [], [], []

    # go through each row of the datafrane (this may be too slow once we do multiple companies )
    for i,v in df.iterrows():
        
        temp_market.append(marketCap(v['open'], v['shareIssued'])) 
        temp_price_per_eps.append(pricePerEarningsPerShare(v['open'], v['basicEPS']))
        temp_price_to_book.append(priceToBook(v['open'], v['netDebt']))
    df.insert(loc = len(df.columns), column='marketCap', value=temp_market)
    df.insert(loc = len(df.columns), column='priceToEarningsRatio', value=temp_price_per_eps)
    df.insert(loc = len(df.columns), column='priceToBook', value=temp_price_to_book)
    return df

# fn to produce dataframe for a company

In [64]:
from yahoofinancials import YahooFinancials

def getDfForCompany(ticker):
    """
    Inputs: ticker of compnay 
    Outputs: A dataframe that contains all the metrics of interest for this project that come from financial statements and price history (ie non calculated metrics)
    """

    yahoo_financials = YahooFinancials(ticker)


    df = getIncomeStatement(ticker, yahoo_financials)
    inc_df = getIncomeMetrics(pd.DataFrame(df))

    df = getBalanceSheet(ticker, yahoo_financials)
    bal_df = getBalanceSheetMetrics(df)

    df = getCashSheet(ticker, yahoo_financials)
    cash_df = getCashMetrics(df)

    price_df = getPrices(ticker, yahoo_financials)

    # merge dataframes
    merge = pd.concat([inc_df, bal_df, cash_df, price_df], axis=1)
    
    # add company ticker to each quarter
    merge["Company"] = [ticker for i in range(merge.__len__())]

    # calculate metrics 
    merge = getCalculatedMetrics(merge)
    return merge


In [65]:
# import time
# start = time.time()
# df = getDfForCompany('GOOG')
# end = time.time()
# print("Function takes ", end-start, "seconds") # ~19.9s as of 4/6 -> 18.79s, 18.27s, 19.4, 18.2 after changing the fns. 
# df

# Test with multiple companies

In [66]:
# list of all companies in the S&P (from wikipedia table)
tickers = [
'MMM',
'AOS',
'ABT',
'ABBV',
'ACN',
'ATVI',
'ADM',
'ADBE',
'ADP',
'AAP',
'AES',
'AFL',
'A',
'APD',
'AKAM',
'ALK',
'ALB',
'ARE',
'ALGN',
'ALLE',
'LNT',
'ALL',
'GOOGL',
'GOOG',
'MO',
'AMZN',
'AMCR',
'AMD',
'AEE',
'AAL',
'AEP',
'AXP',
'AIG',
'AMT',
'AWK',
'AMP',
'ABC',
'AME',
'AMGN',
'APH',
'ADI',
'ANSS',
'AON',
'APA',
'AAPL',
'AMAT',
'APTV',
'ACGL',
'ANET',
'AJG',
'AIZ',
'T',
'ATO',
'ADSK',
'AZO',
'AVB',
'AVY',
'BKR',
'BALL',
'BAC',
'BBWI',
'BAX',
'BDX',
'WRB',
'BRK.B',
'BBY',
'BIO',
'TECH',
'BIIB',
'BLK',
'BK',
'BA',
'BKNG',
'BWA',
'BXP',
'BSX',
'BMY',
'AVGO',
'BR',
'BRO',
'BF.B',
'BG',
'CHRW',
'CDNS',
'CZR',
'CPT',
'CPB',
'COF',
'CAH',
'KMX',
'CCL',
'CARR',
'CTLT',
'CAT',
'CBOE',
'CBRE',
'CDW',
'CE',
'CNC',
'CNP',
'CDAY',
'CF',
'CRL',
'SCHW',
'CHTR',
'CVX',
'CMG',
'CB',
'CHD',
'CI',
'CINF',
'CTAS',
'CSCO',
'C',
'CFG',
'CLX',
'CME',
'CMS',
'KO',
'CTSH',
'CL',
'CMCSA',
'CMA',
'CAG',
'COP',
'ED',
'STZ',
'CEG',
'COO',
'CPRT',
'GLW',
'CTVA',
'CSGP',
'COST',
'CTRA',
'CCI',
'CSX',
'CMI',
'CVS',
'DHI',
'DHR',
'DRI',
'DVA',
'DE',
'DAL',
'XRAY',
'DVN',
'DXCM',
'FANG',
'DLR',
'DFS',
'DISH',
'DIS',
'DG',
'DLTR',
'D',
'DPZ',
'DOV',
'DOW',
'DTE',
'DUK',
'DD',
'DXC',
'EMN',
'ETN',
'EBAY',
'ECL',
'EIX',
'EW',
'EA',
'ELV',
'LLY',
'EMR',
'ENPH',
'ETR',
'EOG',
'EPAM',
'EQT',
'EFX',
'EQIX',
'EQR',
'ESS',
'EL',
'ETSY',
'RE',
'EVRG',
'ES',
'EXC',
'EXPE',
'EXPD',
'EXR',
'XOM',
'FFIV',
'FDS',
'FICO',
'FAST',
'FRT',
'FDX',
'FITB',
'FRC',
'FSLR',
'FE',
'FIS',
'FISV',
'FLT',
'FMC',
'F',
'FTNT',
'FTV',
'FOXA',
'FOX',
'BEN',
'FCX',
'GRMN',
'IT',
'GEHC',
'GEN',
'GNRC',
'GD',
'GE',
'GIS',
'GM',
'GPC',
'GILD',
'GL',
'GPN',
'GS',
'HAL',
'HIG',
'HAS',
'HCA',
'PEAK',
'HSIC',
'HSY',
'HES',
'HPE',
'HLT',
'HOLX',
'HD',
'HON',
'HRL',
'HST',
'HWM',
'HPQ',
'HUM',
'HBAN',
'HII',
'IBM',
'IEX',
'IDXX',
'ITW',
'ILMN',
'INCY',
'IR',
'PODD',
'INTC',
'ICE',
'IFF',
'IP',
'IPG',
'INTU',
'ISRG',
'IVZ',
'INVH',
'IQV',
'IRM',
'JBHT',
'JKHY',
'J',
'JNJ',
'JCI',
'JPM',
'JNPR',
'K',
'KDP',
'KEY',
'KEYS',
'KMB',
'KIM',
'KMI',
'KLAC',
'KHC',
'KR',
'LHX',
'LH',
'LRCX',
'LW',
'LVS',
'LDOS',
'LEN',
'LNC',
'LIN',
'LYV',
'LKQ',
'LMT',
'L',
'LOW',
'LYB',
'MTB',
'MRO',
'MPC',
'MKTX',
'MAR',
'MMC',
'MLM',
'MAS',
'MA',
'MTCH',
'MKC',
'MCD',
'MCK',
'MDT',
'MRK',
'META',
'MET',
'MTD',
'MGM',
'MCHP',
'MU',
'MSFT',
'MAA',
'MRNA',
'MHK',
'MOH',
'TAP',
'MDLZ',
'MPWR',
'MNST',
'MCO',
'MS',
'MOS',
'MSI',
'MSCI',
'NDAQ',
'NTAP',
'NFLX',
'NWL',
'NEM',
'NWSA',
'NWS',
'NEE',
'NKE',
'NI',
'NDSN',
'NSC',
'NTRS',
'NOC',
'NCLH',
'NRG',
'NUE',
'NVDA',
'NVR',
'NXPI',
'ORLY',
'OXY',
'ODFL',
'OMC',
'ON',
'OKE',
'ORCL',
'OGN',
'OTIS',
'PCAR',
'PKG',
'PARA',
'PH',
'PAYX',
'PAYC',
'PYPL',
'PNR',
'PEP',
'PKI',
'PFE',
'PCG',
'PM',
'PSX',
'PNW',
'PXD',
'PNC',
'POOL',
'PPG',
'PPL',
'PFG',
'PG',
'PGR',
'PLD',
'PRU',
'PEG',
'PTC',
'PSA',
'PHM',
'QRVO',
'PWR',
'QCOM',
'DGX',
'RL',
'RJF',
'RTX',
'O',
'REG',
'REGN',
'RF',
'RSG',
'RMD',
'RHI',
'ROK',
'ROL',
'ROP',
'ROST',
'RCL',
'SPGI',
'CRM',
'SBAC',
'SLB',
'STX',
'SEE',
'SRE',
'NOW',
'SHW',
'SPG',
'SWKS',
'SJM',
'SNA',
'SEDG',
'SO',
'LUV',
'SWK',
'SBUX',
'STT',
'STLD',
'STE',
'SYK',
'SYF',
'SNPS',
'SYY',
'TMUS',
'TROW',
'TTWO',
'TPR',
'TRGP',
'TGT',
'TEL',
'TDY',
'TFX',
'TER',
'TSLA',
'TXN',
'TXT',
'TMO',
'TJX',
'TSCO',
'TT',
'TDG',
'TRV',
'TRMB',
'TFC',
'TYL',
'TSN',
'USB',
'UDR',
'ULTA',
'UNP',
'UAL',
'UPS',
'URI',
'UNH',
'UHS',
'VLO',
'VTR',
'VRSN',
'VRSK',
'VZ',
'VRTX',
'VFC',
'VTRS',
'VICI',
'V',
'VMC',
'WAB',
'WBA',
'WMT',
'WBD',
'WM',
'WAT',
'WEC',
'WFC',
'WELL',
'WST',
'WDC',
'WRK',
'WY',
'WHR',
'WMB',
'WTW',
'GWW',
'WYNN',
'XEL',
'XYL',
'YUM',
'ZBRA',
'ZBH',
'ZION',
'ZTS'
]

In [67]:
# import pandas as pd
# import time 
# start = time.time()
# # tickers = ['AAPL', 'GOOG', 'AMZN', 'AAL'] # inital tester list of just 4 companies 
# # list of all companies
# df = getDfForCompany(tickers[0])
# tickers.remove(tickers[0])

# for ticker in tickers:
#     try: 
#         df = pd.concat([df, getDfForCompany(ticker)])
#     except:
#         print("Error for {}".format(ticker))
# end = time.time()
# print("Function takes ", end-start, "seconds") # 91.83s with tester 4 companies
# df

Error for AMD
Error for BRK.B
Error for BF.B
Error for CTLT
Error for COP
Error for GEHC
Error for JPM
Error for UAL
Function takes  17735.935105085373 seconds


,dilutedNIAvailtoComStockholders,grossProfit,netIncome,totalExpenses,totalRevenue,basicEPS,cashEquivalents,currentAssets,currentDebt,netDebt,...,high,low,open,close,volume,adjclose,Company,marketCap,priceToEarningsRatio,priceToBook
2022-03-31,1299000000.0,4003000000.0,1299000000.0,7121000000.0,8829000000.0,2.27,NaN,14452000000.0,1877000000.0,13431000000.0,...,151.250000,148.800003,150.449997,148.880005,3193800.0,142.320374,MMM,1.420298e+11,66.277532,1.120170e-08
2022-06-30,78000000.0,3609000000.0,78000000.0,8592000000.0,8702000000.0,0.14,NaN,14514000000.0,2257000000.0,13554000000.0,...,130.210007,127.300003,128.710007,129.410004,3341100.0,124.958389,MMM,1.215065e+11,919.357191,9.496090e-09
2022-09-30,3859000000.0,3891000000.0,3859000000.0,7187000000.0,8619000000.0,6.79,NaN,14895000000.0,1856000000.0,12301000000.0,...,113.150002,110.389999,112.000000,110.500000,3112800.0,107.793869,MMM,6.190721e+10,16.494845,9.104951e-09
2022-12-31,541000000.0,14997000000.0,5777000000.0,29895000000.0,8079000000.0,0.98,NaN,14688000000.0,1938000000.0,12284000000.0,...,120.029999,118.510002,119.650002,119.919998,2096000.0,118.352959,MMM,1.129536e+11,122.091838,9.740313e-09
2023-03-31,976000000.0,14412000000.0,5454000000.0,29533000000.0,8031000000.0,1.77,NaN,14963000000.0,3012000000.0,12136000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,MMM,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-31,875000000.0,<NA>,908000000.0,<NA>,3305000000.0,1.33,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ZION,NaN,NaN,NaN
2022-03-31,595000000.0,1417000000.0,595000000.0,1197000000.0,1986000000.0,1.26,NaN,6837000000.0,1350000000.0,3443000000.0,...,194.899994,188.490005,192.449997,188.589996,2477600.0,186.676575,ZTS,9.658897e+10,152.738093,5.589602e-08
2022-06-30,529000000.0,1427000000.0,529000000.0,1326000000.0,2052000000.0,1.13,NaN,6651000000.0,1352000000.0,3925000000.0,...,173.550003,169.830002,172.440002,171.889999,2220700.0,170.443924,ZTS,8.654613e+10,152.601772,4.393376e-08
2022-09-30,529000000.0,1395000000.0,529000000.0,1279000000.0,2002000000.0,1.13,NaN,6551000000.0,1353000000.0,4060000000.0,...,152.029999,148.039993,150.419998,148.289993,2437200.0,147.312302,ZTS,7.549448e+10,133.115043,3.704926e-08


# Export Data

In [68]:

# df.to_csv('/Applications/Repos/value-investing/company-data.csv') # Export tester data to csv to start analysis 
# df.to_csv('/Applications/Repos/value-investing/sp500-company-data.csv') # all companies 